In [12]:
import os
import pandas as pd

In [13]:
# subject info data
info_path = '../data/processed/subject_info.csv'
subject_info = pd.read_csv(info_path)

In [14]:
subject_info.head()

,subject,Age,Height (cm),Weight (kg),Gender,Dominant hand,Did you drink coffee today?,Did you drink coffee within the last hour?,Did you do any sports today?,Are you a smoker?,Did you smoke within the last hour?,Do you feel ill today?
0,S2,27,175,80,male,right,NO,NO,NO,NO,NO,NO
1,S3,27,173,69,male,right,NO,NO,NO,NO,NO,NO
2,S4,25,175,90,male,right,NO,NO,NO,NO,NO,NO
3,S5,35,189,80,male,right,YES,NO,NO,NO,NO,NO
4,S6,27,170,66,male,right,YES,NO,NO,YES,NO,NO


In [15]:
# E4 data
main_e4_path = '../data/processed/apple_watch_format/'
subject = 'S2'
e4_path = f'{subject}_apple_watch.csv'

In [16]:
full_e4_path = os.path.join(main_e4_path, e4_path)
e4_data = pd.read_csv(full_e4_path)

In [18]:
e4_data['label'].value_counts()

label
0    52
1    22
Name: count, dtype: int64

In [19]:
# Balanced data
focus = e4_data[e4_data["label"] == 1]
non_focus = e4_data[e4_data["label"] == 0].sample(n=len(focus), random_state=42)
e4_balanced = pd.concat([focus, non_focus]).sample(frac=1, random_state=42)

In [20]:
e4_balanced['label'].value_counts()

label
0    22
1    22
Name: count, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split

X = e4_balanced.drop(columns=["label", "subject"])
y = e4_balanced["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [28]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

/opt/homebrew/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:32:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [30]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.75      0.86         4

    accuracy                           0.89         9
   macro avg       0.92      0.88      0.88         9
weighted avg       0.91      0.89      0.89         9



In [31]:
sorted(zip(X.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)

[('ACC_x_mean', np.float32(1.0)),
 ('ACC_y_mean', np.float32(0.0)),
 ('ACC_z_mean', np.float32(0.0)),
 ('net_acc_mean', np.float32(0.0)),
 ('net_acc_std', np.float32(0.0)),
 ('HR', np.float32(0.0)),
 ('IBI', np.float32(0.0)),
 ('HRV', np.float32(0.0))]

# Train xgboost with all subject

In [41]:
import pandas as pd
import glob

In [42]:
all_files = glob.glob('../data/processed/apple_watch_format/S*_apple_watch.csv')

In [43]:
df_all = pd.concat([pd.read_csv(file) for file in all_files], axis=0, ignore_index=True)

In [44]:
df_all

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,label,subject
0,-46.582292,-6.051042,32.276042,64.614681,4.654549,69.224135,1152.500000,619.704125,0,4
1,-43.976042,-31.740625,-19.667708,64.940738,4.375065,79.507543,915.322581,738.055935,0,4
2,-56.694792,-7.480208,-23.934375,65.363949,1.140627,72.979079,1045.138889,753.696819,0,4
3,-33.206250,-8.705208,-54.866667,64.771843,0.621698,78.275497,1008.620690,855.633087,0,4
4,-33.027083,-8.196875,-54.929167,64.693208,0.778812,68.677229,1160.156250,854.699485,0,4
...,...,...,...,...,...,...,...,...,...,...
1145,-19.776042,-15.045833,38.236458,63.668923,4.141692,88.756221,754.807692,334.043583,0,11
1146,-6.693750,-31.478125,44.279167,62.952634,2.308099,92.192958,756.410256,424.894772,0,11
1147,17.828125,-51.720833,28.890625,62.335270,0.898838,80.435572,858.901515,471.185081,0,11
1148,6.482292,-54.142708,29.834375,62.491297,1.131574,84.352701,820.312500,472.101117,0,11


In [45]:
print(df_all.value_counts('subject'))

subject
11    79
10    78
13    78
14    78
15    78
17    78
8     77
3     76
5     76
6     76
7     76
16    76
4     75
9     75
2     74
Name: count, dtype: int64


In [46]:
print(df_all.value_counts('label'))

label
0    803
1    347
Name: count, dtype: int64
